## Mini Project | Supervised Learning and Ensembles
----
#### Submitted By: GROUP 5

**Team Members**

> Sandeep Keswani

> Vivek V Krishnan

> Seeju Kumar 

> Shashank Shekhar

In [4]:
%matplotlib inline

In [5]:
import pandas as pd

In [6]:
import sklearn as sk

In [7]:
import seaborn as sns

In [8]:
import matplotlib.pyplot as plt

In [9]:
import numpy as np

In [10]:
from scipy.stats import zscore

In [3]:
pd.read_excel("data/Bank_Personal_Loan_Modelling-1.xlsx", sheet_name=1)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.600000,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.500000,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.000000,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.700000,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.000000,2,0,0,0,0,0,1
5,6,37,13,29,92121,4,0.400000,2,155,0,0,0,1,0
6,7,53,27,72,91711,2,1.500000,2,0,0,0,0,1,0
7,8,50,24,22,93943,1,0.300000,3,0,0,0,0,0,1
8,9,35,10,81,90089,3,0.600000,2,104,0,0,0,1,0
9,10,34,9,180,93023,1,8.900000,3,0,1,0,0,0,0
